In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

## read reverse mortgage data

In [2]:
alameda_rm_all = pd.read_csv('./reverse_mortgage_data/alameda_rm_all.csv', dtype={
    'county_code': str,
    'census_tract': str
})
alameda_rm_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3753 entries, 0 to 3752
Data columns (total 99 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   activity_year                             3753 non-null   int64  
 1   lei                                       3753 non-null   object 
 2   derived_msa-md                            3753 non-null   int64  
 3   state_code                                3753 non-null   object 
 4   county_code                               3753 non-null   object 
 5   census_tract                              3752 non-null   object 
 6   conforming_loan_limit                     3753 non-null   object 
 7   derived_loan_product_type                 3753 non-null   object 
 8   derived_dwelling_category                 3753 non-null   object 
 9   derived_ethnicity                         3753 non-null   object 
 10  derived_race                        

## read census data and filter to county

In [3]:
census_all = pd.read_csv('./census_data/ACS2019_filtered0623.csv', dtype={
    'census_tract': str,
    'county_code': str,
    'census_tract_np': str,
    'census_tract_period': str
})
census_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4505 entries, 0 to 4504
Columns: 208 entries, census_tract to Percent!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female
dtypes: float64(3), int64(105), object(100)
memory usage: 7.1+ MB


In [4]:
census_alameda = census_all[census_all['county_code'] == '06001'].copy()

In [5]:
census_alameda.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 361 entries, 0 to 360
Columns: 208 entries, census_tract to Percent!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female
dtypes: float64(3), int64(105), object(100)
memory usage: 589.4+ KB


In [34]:
census_alameda.head()

,census_tract,county_code,county,state_code,census_tract_np,census_tract_period,total_population,62yr_population,62yr_pp_pct,median_age,hispanic_population,non_hispanic_population,non_hispanic_white_population,non_hispanic_black_population,non_hispanic_asian_population,other_races_population,hispanic_pp_pct,non_hispanic_pp_pct,non_hispanic_white_pp_pct,non_hispanic_black_pp_pct,non_hispanic_asian_pp_pct,other_races_pp_pct,Unnamed: 22,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!American Indian and Alaska Native alone,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Some other race alone,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races,Unnamed: 27,asian_1_population,black_1_population,Unnamed: 30,Estimate!!SEX AND AGE!!Total population!!Male,Estimate!!SEX AND AGE!!Total population!!Female,Estimate!!SEX AND AGE!!Total population!!Sex ratio (males per 100 females),Estimate!!SEX AND AGE!!Total population!!Under 5 years,Estimate!!SEX AND AGE!!Total population!!5 to 9 years,Estimate!!SEX AND AGE!!Total population!!10 to 14 years,Estimate!!SEX AND AGE!!Total population!!15 to 19 years,Estimate!!SEX AND AGE!!Total population!!20 to 24 years,Estimate!!SEX AND AGE!!Total population!!25 to 34 years,Estimate!!SEX AND AGE!!Total population!!35 to 44 years,Estimate!!SEX AND AGE!!Total population!!45 to 54 years,Estimate!!SEX AND AGE!!Total population!!55 to 59 years,Estimate!!SEX AND AGE!!Total population!!60 to 64 years,Estimate!!SEX AND AGE!!Total population!!65 to 74 years,Estimate!!SEX AND AGE!!Total population!!75 to 84 years,Estimate!!SEX AND AGE!!Total population!!85 years and over,Estimate!!SEX AND AGE!!Total population!!Median age (years),Estimate!!SEX AND AGE!!Total population!!Under 18 years,Estimate!!SEX AND AGE!!Total population!!16 years and over,Estimate!!SEX AND AGE!!Total population!!18 years and over,Estimate!!SEX AND AGE!!Total population!!21 years and over,Estimate!!SEX AND AGE!!Total population!!62 years and over,Estimate!!SEX AND AGE!!Total population!!65 years and over,Estimate!!SEX AND AGE!!Total population!!18 years and over.1,Estimate!!SEX AND AGE!!Total population!!18 years and over!!Male,Estimate!!SEX AND AGE!!Total population!!18 years and over!!Female,Estimate!!SEX AND AGE!!Total population!!18 years and over!!Sex ratio (males per 100 females),Estimate!!SEX AND AGE!!Total population!!65 years and over.1,Estimate!!SEX AND AGE!!Total population!!65 years and over!!Male,Estimate!!SEX AND AGE!!Total population!!65 years and over!!Female,Estimate!!SEX AND AGE!!Total population!!65 years and over!!Sex ratio (males per 100 females),Estimate!!RACE!!Total population,Estimate!!RACE!!Total population!!One race,Estimate!!RACE!!Total population!!Two or more races,Estimate!!RACE!!Total population!!One race.1,Estimate!!RACE!!Total population!!One race!!White,Estimate!!RACE!!Total population!!One race!!Black or African American,Estimate!!RACE!!Total population!!One race!!American Indian and Alaska Native,Estimate!!RACE!!Total population!!One race!!American Indian and Alaska Native!!Cherokee tribal grouping,Estimate!!RACE!!Total population!!One race!!American Indian and Alaska Native!!Chippewa tribal grouping,Estimate!!RACE!!Total population!!One race!!American Indian and Alaska Native!!Navajo tribal grouping,Estimate!!RACE!!Total population!!One race!!American Indian and Alaska Native!!Sioux tribal grouping,Estimate!!RACE!!Total population!!One race!!Asian,Estimate!!RACE!!Total population!!One race!!Asian!!Asian Indian,Estimate!!RACE!!Total population!!One race!!Asian!!Chinese,Estimate!!RACE!!Total population!!One race!!Asian!!Filipino,Estimate!!RACE!!Total population!!One race!!Asian!!Japanese,Estimate!!RACE!!Total population!!One race!!Asian!!Korean,Estimate!!RACE!!Total population

## merge lender list

In [6]:
# read lender list

lender_list_2018 = pd.read_csv('./raw_data/lei_agency_name_2018.csv')

In [7]:
lender_list_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5735 entries, 0 to 5734
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   lei                  5735 non-null   object
 1   activity_year        5735 non-null   int64 
 2   calendar_quarter     5735 non-null   int64 
 3   tax_id               5735 non-null   object
 4   agency_code          5735 non-null   int64 
 5   respondent_name      5735 non-null   object
 6   respondent_state     5735 non-null   object
 7   respondent_city      5735 non-null   object
 8   respondent_zip_code  5735 non-null   object
 9   lar_count            5735 non-null   int64 
dtypes: int64(4), object(6)
memory usage: 448.2+ KB


In [8]:
# merge by lei

alameda_rm_all

,activity_year,lei,derived_msa-md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,derived_race,derived_sex,action_taken,purchaser_type,preapproval,loan_type,loan_purpose,lien_status,reverse_mortgage,open-end_line_of_credit,business_or_commercial_purpose,loan_amount,loan_to_value_ratio,interest_rate,rate_spread,hoepa_status,total_loan_costs,total_points_and_fees,origination_charges,discount_points,lender_credits,loan_term,prepayment_penalty_term,intro_rate_period,negative_amortization,interest_only_payment,balloon_payment,other_nonamortizing_features,property_value,construction_method,occupancy_type,manufactured_home_secured_property_type,manufactured_home_land_property_interest,total_units,multifamily_affordable_units,income,debt_to_income_ratio,applicant_credit_score_type,co-applicant_credit_score_type,applicant_ethnicity-1,applicant_ethnicity-2,applicant_ethnicity-3,applicant_ethnicity-4,applicant_ethnicity-5,co-applicant_ethnicity-1,co-applicant_ethnicity-2,co-applicant_ethnicity-3,co-applicant_ethnicity-4,co-applicant_ethnicity-5,applicant_ethnicity_observed,co-applicant_ethnicity_observed,applicant_race-1,applicant_race-2,applicant_race-3,applicant_race-4,applicant_race-5,co-applicant_race-1,co-applicant_race-2,co-applicant_race-3,co-applicant_race-4,co-applicant_race-5,applicant_race_observed,co-applicant_race_observed,applicant_sex,co-applicant_sex,applicant_sex_observed,co-applicant_sex_observed,applicant_age,co-applicant_age,applicant_age_above_62,co-applicant_age_above_62,submission_of_application,initially_payable_to_institution,aus-1,aus-2,aus-3,aus-4,aus-5,denial_reason-1,denial_reason-2,denial_reason-3,denial_reason-4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
0,2018,549300WH9FMXOC63ZV77,36084,CA,06001,06001408500,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,Black or African American,Female,4,0,2,2,31,1,1,2,2,235000.0,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2,2,NaN,1,1,3,5,1,NaN,0.0,NaN,9,9,2.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,2,4,3.0,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,2,4,2,5,2,4,>74,9999,Yes,NaN,1,1,6,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,5660,98.80,104400,53,560,1656,63
1,2018,549300WH9FMXOC63ZV77,36084,CA,06001,06001400900,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,Black or African American,Male,4,0,2,2,31,1,1,1,2,355000.0,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,2,2,2,2,NaN,1,1,3,5,1,NaN,13.0,NaN,9,9,2.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,3,4,3.0,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,3,4,1,5,3,4,65-74,9999,Yes,NaN,1,1,6,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,2175,57.84,104400,80,296,1050,76
2,2018,549300WH9FMXOC63ZV77,36084,CA,06001,06001425101,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,Race Not Available,Sex Not Available,5,0,2,2,31,1,1,1,2,175000.0,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,2,2,2,2,NaN,1,1,3,5,1,NaN,0.0,NaN,9,9,3.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,3,4,6.0,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,3,4,3,5,3,4,65-74,9999,Yes,NaN,1,1,6,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,1582,39.25,104400,87,686,150,40
3,2018,549300WH9FMXOC63ZV77,36084,CA,06001,06001433400,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,White,Female,4,0,2,2,31,1,1,2,2,305000.0,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2,2,NaN,1,1,3,5,1,NaN,24.0,NaN,9,9,2.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,3,4,5.0,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,3,4,2,5,3,4,>74,9999,Yes,NaN,1,1,6,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,6593,80.48,104400,92,1737,2238,32
4,2018,549300WH9FMXOC63ZV77,36084,CA,06001,06001441921,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,White,Female,1,71,2,2,31,1,1,1,2,285000.0,NaN,5.291,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13

In [9]:
lender_list_2018

,lei,activity_year,calendar_quarter,tax_id,agency_code,respondent_name,respondent_state,respondent_city,respondent_zip_code,lar_count
0,213800QUAI2VH5YM6310,2018,4,05-0402708,7,"Embrace Home Loans, Inc.",RI,Middletown,02842,38810
1,5493001SXWZ4OFP8Z903,2018,4,74-2853239,7,DHI Mortgage Company Limited,TX,Austin,78750,45691
2,549300YHR41F57LS0R28,2018,4,39-0273630,3,Farmers and Merchants Union Bank,WI,Columbus,53925,131
3,549300SX6THT96LFD245,2018,4,26-1866970,3,Oakworth Capital Bank,AL,Birmingham,35209,29
4,2549000UUHZDEKIZ2Z84,2018,4,34-0144365,1,The Citizens National Bank,OH,Woodsfield,43793,97
...,...,...,...,...,...,...,...,...,...,...
5730,Z867SNMO7WGY8TTGQG78,2018,4,55-0744997,3,SUMMIT COMMUNITY BANK,WV,MOOREFIELD,26836,1035
5731,ZF85QS7OXKPBG52R7N18,2018,4,39-1941673,9,Associated Bank NA,WI,Green Bay,54301,20022
5732,ZXMJHJK466PBZTM5F379,2018,4,59-3244348,9,Raymond James Bank National Association,FL,SAINT PETERSBURG,33716,2067
5733,ZXZDWBXF4U47HEP2EO46,2018,4,33-0244449,2,Silvergate Bank,CA,La Jolla,92037,298


In [10]:
alameda_rm_full = pd.merge(
    alameda_rm_all,
    lender_list_2018,
    on='lei',
    how='left',
    validate='m:1'
)

In [11]:
alameda_rm_full.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3753 entries, 0 to 3752
Data columns (total 108 columns):
 #    Column                                    Non-Null Count  Dtype  
---   ------                                    --------------  -----  
 0    activity_year_x                           3753 non-null   int64  
 1    lei                                       3753 non-null   object 
 2    derived_msa-md                            3753 non-null   int64  
 3    state_code                                3753 non-null   object 
 4    county_code                               3753 non-null   object 
 5    census_tract                              3752 non-null   object 
 6    conforming_loan_limit                     3753 non-null   object 
 7    derived_loan_product_type                 3753 non-null   object 
 8    derived_dwelling_category                 3753 non-null   object 
 9    derived_ethnicity                         3753 non-null   object 
 10   derived_race          

In [12]:
alameda_rm_name = alameda_rm_full.drop(columns= ['activity_year_y', 'calendar_quarter', 'lar_count']).copy()

In [13]:
alameda_rm_name.head()

,activity_year_x,lei,derived_msa-md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,derived_race,derived_sex,action_taken,purchaser_type,preapproval,loan_type,loan_purpose,lien_status,reverse_mortgage,open-end_line_of_credit,business_or_commercial_purpose,loan_amount,loan_to_value_ratio,interest_rate,rate_spread,hoepa_status,total_loan_costs,total_points_and_fees,origination_charges,discount_points,lender_credits,loan_term,prepayment_penalty_term,intro_rate_period,negative_amortization,interest_only_payment,balloon_payment,other_nonamortizing_features,property_value,construction_method,occupancy_type,manufactured_home_secured_property_type,manufactured_home_land_property_interest,total_units,multifamily_affordable_units,income,debt_to_income_ratio,applicant_credit_score_type,co-applicant_credit_score_type,applicant_ethnicity-1,applicant_ethnicity-2,applicant_ethnicity-3,applicant_ethnicity-4,applicant_ethnicity-5,co-applicant_ethnicity-1,co-applicant_ethnicity-2,co-applicant_ethnicity-3,co-applicant_ethnicity-4,co-applicant_ethnicity-5,applicant_ethnicity_observed,co-applicant_ethnicity_observed,applicant_race-1,applicant_race-2,applicant_race-3,applicant_race-4,applicant_race-5,co-applicant_race-1,co-applicant_race-2,co-applicant_race-3,co-applicant_race-4,co-applicant_race-5,applicant_race_observed,co-applicant_race_observed,applicant_sex,co-applicant_sex,applicant_sex_observed,co-applicant_sex_observed,applicant_age,co-applicant_age,applicant_age_above_62,co-applicant_age_above_62,submission_of_application,initially_payable_to_institution,aus-1,aus-2,aus-3,aus-4,aus-5,denial_reason-1,denial_reason-2,denial_reason-3,denial_reason-4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units,tax_id,agency_code,respondent_name,respondent_state,respondent_city,respondent_zip_code
0,2018,549300WH9FMXOC63ZV77,36084,CA,06001,06001408500,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,Black or African American,Female,4,0,2,2,31,1,1,2,2,235000.0,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2,2,NaN,1,1,3,5,1,NaN,0.0,NaN,9,9,2.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,2,4,3.0,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,2,4,2,5,2,4,>74,9999,Yes,NaN,1,1,6,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,5660,98.80,104400,53,560,1656,63,11-3469077,7,Nationwide Equities Corporation,NJ,Mahwah,07495
1,2018,549300WH9FMXOC63ZV77,36084,CA,06001,06001400900,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,Black or African American,Male,4,0,2,2,31,1,1,1,2,355000.0,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,2,2,2,2,NaN,1,1,3,5,1,NaN,13.0,NaN,9,9,2.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,3,4,3.0,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,3,4,1,5,3,4,65-74,9999,Yes,NaN,1,1,6,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,2175,57.84,104400,80,296,1050,76,11-3469077,7,Nationwide Equities Corporation,NJ,Mahwah,07495
2,2018,549300WH9FMXOC63ZV77,36084,CA,06001,06001425101,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,Race Not Available,Sex Not Available,5,0,2,2,31,1,1,1,2,175000.0,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,2,2,2,2,NaN,1,1,3,5,1,NaN,0.0,NaN,9,9,3.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,3,4,6.0,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,3,4,3,5,3,4,65-74,9999,Yes,NaN,1,1,6,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,1582,39.25,104400,87,686,150,40,11-3469077,7,Nationwide Equities Corporation,NJ,Mahwah,07495
3,2018,549300WH9FMXOC63ZV77,36084,CA,06001,06001433400,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,White,Female,4,0,2,2,31,1,1,2,2,305000.0,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2,2,NaN,1,1,3,5,1,NaN,24.0,NaN,9,9,2.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,3,4,5.0,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,3,4,2,5,3,4,>74,9999,Yes,NaN,1,1,6,NaN,NaN,NaN,N

## group by census tracts

In [14]:
# filter by action_taken = 1

alameda_rm_originated = alameda_rm_name[alameda_rm_name['action_taken'] == 1]

In [15]:
alameda_rm_originated.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2078 entries, 4 to 3751
Data columns (total 105 columns):
 #    Column                                    Non-Null Count  Dtype  
---   ------                                    --------------  -----  
 0    activity_year_x                           2078 non-null   int64  
 1    lei                                       2078 non-null   object 
 2    derived_msa-md                            2078 non-null   int64  
 3    state_code                                2078 non-null   object 
 4    county_code                               2078 non-null   object 
 5    census_tract                              2078 non-null   object 
 6    conforming_loan_limit                     2078 non-null   object 
 7    derived_loan_product_type                 2078 non-null   object 
 8    derived_dwelling_category                 2078 non-null   object 
 9    derived_ethnicity                         2078 non-null   object 
 10   derived_race          

In [16]:
# group by census_tract

alameda_census_counts = alameda_rm_originated.groupby(['census_tract']).count()

In [17]:
alameda_census_counts = alameda_census_counts[['lei']].copy()

In [18]:
alameda_census_counts = alameda_census_counts.reset_index()

In [19]:
alameda_census_counts = alameda_census_counts.sort_values(by=['lei'], ascending=False).copy()
alameda_census_counts.rename(columns={'lei': 'rm_no'}, inplace=True)

In [20]:
alameda_census_counts = alameda_census_counts.reset_index(drop=True)

In [21]:
alameda_census_counts

,census_tract,rm_no
0,06001450602,25
1,06001433300,24
2,06001407700,21
3,06001408100,21
4,06001444100,20
...,...,...
308,06001436800,1
309,06001423902,1
310,06001403400,1
311,06001403100,1


## merge with census data

In [22]:
census_alameda

,census_tract,county_code,county,state_code,census_tract_np,census_tract_period,total_population,62yr_population,62yr_pp_pct,median_age,hispanic_population,non_hispanic_population,non_hispanic_white_population,non_hispanic_black_population,non_hispanic_asian_population,other_races_population,hispanic_pp_pct,non_hispanic_pp_pct,non_hispanic_white_pp_pct,non_hispanic_black_pp_pct,non_hispanic_asian_pp_pct,other_races_pp_pct,Unnamed: 22,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!American Indian and Alaska Native alone,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Some other race alone,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races,Unnamed: 27,asian_1_population,black_1_population,Unnamed: 30,Estimate!!SEX AND AGE!!Total population!!Male,Estimate!!SEX AND AGE!!Total population!!Female,Estimate!!SEX AND AGE!!Total population!!Sex ratio (males per 100 females),Estimate!!SEX AND AGE!!Total population!!Under 5 years,Estimate!!SEX AND AGE!!Total population!!5 to 9 years,Estimate!!SEX AND AGE!!Total population!!10 to 14 years,Estimate!!SEX AND AGE!!Total population!!15 to 19 years,Estimate!!SEX AND AGE!!Total population!!20 to 24 years,Estimate!!SEX AND AGE!!Total population!!25 to 34 years,Estimate!!SEX AND AGE!!Total population!!35 to 44 years,Estimate!!SEX AND AGE!!Total population!!45 to 54 years,Estimate!!SEX AND AGE!!Total population!!55 to 59 years,Estimate!!SEX AND AGE!!Total population!!60 to 64 years,Estimate!!SEX AND AGE!!Total population!!65 to 74 years,Estimate!!SEX AND AGE!!Total population!!75 to 84 years,Estimate!!SEX AND AGE!!Total population!!85 years and over,Estimate!!SEX AND AGE!!Total population!!Median age (years),Estimate!!SEX AND AGE!!Total population!!Under 18 years,Estimate!!SEX AND AGE!!Total population!!16 years and over,Estimate!!SEX AND AGE!!Total population!!18 years and over,Estimate!!SEX AND AGE!!Total population!!21 years and over,Estimate!!SEX AND AGE!!Total population!!62 years and over,Estimate!!SEX AND AGE!!Total population!!65 years and over,Estimate!!SEX AND AGE!!Total population!!18 years and over.1,Estimate!!SEX AND AGE!!Total population!!18 years and over!!Male,Estimate!!SEX AND AGE!!Total population!!18 years and over!!Female,Estimate!!SEX AND AGE!!Total population!!18 years and over!!Sex ratio (males per 100 females),Estimate!!SEX AND AGE!!Total population!!65 years and over.1,Estimate!!SEX AND AGE!!Total population!!65 years and over!!Male,Estimate!!SEX AND AGE!!Total population!!65 years and over!!Female,Estimate!!SEX AND AGE!!Total population!!65 years and over!!Sex ratio (males per 100 females),Estimate!!RACE!!Total population,Estimate!!RACE!!Total population!!One race,Estimate!!RACE!!Total population!!Two or more races,Estimate!!RACE!!Total population!!One race.1,Estimate!!RACE!!Total population!!One race!!White,Estimate!!RACE!!Total population!!One race!!Black or African American,Estimate!!RACE!!Total population!!One race!!American Indian and Alaska Native,Estimate!!RACE!!Total population!!One race!!American Indian and Alaska Native!!Cherokee tribal grouping,Estimate!!RACE!!Total population!!One race!!American Indian and Alaska Native!!Chippewa tribal grouping,Estimate!!RACE!!Total population!!One race!!American Indian and Alaska Native!!Navajo tribal grouping,Estimate!!RACE!!Total population!!One race!!American Indian and Alaska Native!!Sioux tribal grouping,Estimate!!RACE!!Total population!!One race!!Asian,Estimate!!RACE!!Total population!!One race!!Asian!!Asian Indian,Estimate!!RACE!!Total population!!One race!!Asian!!Chinese,Estimate!!RACE!!Total population!!One race!!Asian!!Filipino,Estimate!!RACE!!Total population!!One race!!Asian!!Japanese,Estimate!!RACE!!Total population!!One race!!Asian!!Korean,Estimate!!RACE!!Total population

In [23]:
alameda_census_rm = pd.merge(
    alameda_census_counts,
    census_alameda,
    on='census_tract',
    how='left',
    validate='1:1'
)

In [24]:
alameda_census_rm.sample(10)

,census_tract,rm_no,county_code,county,state_code,census_tract_np,census_tract_period,total_population,62yr_population,62yr_pp_pct,median_age,hispanic_population,non_hispanic_population,non_hispanic_white_population,non_hispanic_black_population,non_hispanic_asian_population,other_races_population,hispanic_pp_pct,non_hispanic_pp_pct,non_hispanic_white_pp_pct,non_hispanic_black_pp_pct,non_hispanic_asian_pp_pct,other_races_pp_pct,Unnamed: 22,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!American Indian and Alaska Native alone,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Some other race alone,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races,Unnamed: 27,asian_1_population,black_1_population,Unnamed: 30,Estimate!!SEX AND AGE!!Total population!!Male,Estimate!!SEX AND AGE!!Total population!!Female,Estimate!!SEX AND AGE!!Total population!!Sex ratio (males per 100 females),Estimate!!SEX AND AGE!!Total population!!Under 5 years,Estimate!!SEX AND AGE!!Total population!!5 to 9 years,Estimate!!SEX AND AGE!!Total population!!10 to 14 years,Estimate!!SEX AND AGE!!Total population!!15 to 19 years,Estimate!!SEX AND AGE!!Total population!!20 to 24 years,Estimate!!SEX AND AGE!!Total population!!25 to 34 years,Estimate!!SEX AND AGE!!Total population!!35 to 44 years,Estimate!!SEX AND AGE!!Total population!!45 to 54 years,Estimate!!SEX AND AGE!!Total population!!55 to 59 years,Estimate!!SEX AND AGE!!Total population!!60 to 64 years,Estimate!!SEX AND AGE!!Total population!!65 to 74 years,Estimate!!SEX AND AGE!!Total population!!75 to 84 years,Estimate!!SEX AND AGE!!Total population!!85 years and over,Estimate!!SEX AND AGE!!Total population!!Median age (years),Estimate!!SEX AND AGE!!Total population!!Under 18 years,Estimate!!SEX AND AGE!!Total population!!16 years and over,Estimate!!SEX AND AGE!!Total population!!18 years and over,Estimate!!SEX AND AGE!!Total population!!21 years and over,Estimate!!SEX AND AGE!!Total population!!62 years and over,Estimate!!SEX AND AGE!!Total population!!65 years and over,Estimate!!SEX AND AGE!!Total population!!18 years and over.1,Estimate!!SEX AND AGE!!Total population!!18 years and over!!Male,Estimate!!SEX AND AGE!!Total population!!18 years and over!!Female,Estimate!!SEX AND AGE!!Total population!!18 years and over!!Sex ratio (males per 100 females),Estimate!!SEX AND AGE!!Total population!!65 years and over.1,Estimate!!SEX AND AGE!!Total population!!65 years and over!!Male,Estimate!!SEX AND AGE!!Total population!!65 years and over!!Female,Estimate!!SEX AND AGE!!Total population!!65 years and over!!Sex ratio (males per 100 females),Estimate!!RACE!!Total population,Estimate!!RACE!!Total population!!One race,Estimate!!RACE!!Total population!!Two or more races,Estimate!!RACE!!Total population!!One race.1,Estimate!!RACE!!Total population!!One race!!White,Estimate!!RACE!!Total population!!One race!!Black or African American,Estimate!!RACE!!Total population!!One race!!American Indian and Alaska Native,Estimate!!RACE!!Total population!!One race!!American Indian and Alaska Native!!Cherokee tribal grouping,Estimate!!RACE!!Total population!!One race!!American Indian and Alaska Native!!Chippewa tribal grouping,Estimate!!RACE!!Total population!!One race!!American Indian and Alaska Native!!Navajo tribal grouping,Estimate!!RACE!!Total population!!One race!!American Indian and Alaska Native!!Sioux tribal grouping,Estimate!!RACE!!Total population!!One race!!Asian,Estimate!!RACE!!Total population!!One race!!Asian!!Asian Indian,Estimate!!RACE!!Total population!!One race!!Asian!!Chinese,Estimate!!RACE!!Total population!!One race!!Asian!!Filipino,Estimate!!RACE!!Total population!!One race!!Asian!!Japanese,Estimate!!RACE!!Total population!!One race!!Asian!!Korean,Estimate!!RACE!!Total popu

In [25]:
alameda_census_rm.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 313 entries, 0 to 312
Data columns (total 209 columns):
 #    Column                                                                                                                                                            Dtype  
---   ------                                                                                                                                                            -----  
 0    census_tract                                                                                                                                                      object 
 1    rm_no                                                                                                                                                             int64  
 2    county_code                                                                                                                                                       object 
 3    county  

In [26]:
# subset to the columns that I need

alameda_census_rm = alameda_census_rm[['census_tract', 'rm_no', 'county_code', 'county', 'state_code', 'census_tract_np', 'census_tract_period', 'total_population', '62yr_pp_pct', 'median_age', 'hispanic_pp_pct', 'non_hispanic_black_pp_pct']].copy()

In [27]:
alameda_census_rm

,census_tract,rm_no,county_code,county,state_code,census_tract_np,census_tract_period,total_population,62yr_pp_pct,median_age,hispanic_pp_pct,non_hispanic_black_pp_pct
0,06001450602,25,06001,Alameda County,CA,450602,4506.02,10265,18.9%,43.4,8.2%,2.7%
1,06001433300,24,06001,Alameda County,CA,433300,4333,7761,20.9%,42.8,23.5%,0.3%
2,06001407700,21,06001,Alameda County,CA,407700,4077,4122,19.5%,43.9,16.2%,42.9%
3,06001408100,21,06001,Alameda County,CA,408100,4081,6320,28.7%,45,10.1%,23.8%
4,06001444100,20,06001,Alameda County,CA,444100,4441,7556,24.2%,46.9,28.8%,1.1%
...,...,...,...,...,...,...,...,...,...,...,...,...
308,06001436800,1,06001,Alameda County,CA,436800,4368,4382,12.7%,35.6,45.9%,13.5%
309,06001423902,1,06001,Alameda County,CA,423902,4239.02,1605,21.3%,39.5,6.0%,2.9%
310,06001403400,1,06001,Alameda County,CA,403400,4034,4340,22.8%,36.3,8.2%,19.1%
311,06001403100,1,06001,Alameda County,CA,403100,4031,2101,18.1%,36.5,13.0%,17.7%


In [28]:
# calculate rm per 1000 people

alameda_census_rm['_rm_per_1000'] = (
    alameda_census_rm['rm_no'] / alameda_census_rm['total_population'] * 1000
)

In [29]:
alameda_census_rm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 313 entries, 0 to 312
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   census_tract               313 non-null    object 
 1   rm_no                      313 non-null    int64  
 2   county_code                313 non-null    object 
 3   county                     313 non-null    object 
 4   state_code                 313 non-null    object 
 5   census_tract_np            313 non-null    object 
 6   census_tract_period        313 non-null    object 
 7   total_population           313 non-null    int64  
 8   62yr_pp_pct                313 non-null    object 
 9   median_age                 313 non-null    object 
 10  hispanic_pp_pct            313 non-null    object 
 11  non_hispanic_black_pp_pct  313 non-null    object 
 12  _rm_per_1000               313 non-null    float64
dtypes: float64(1), int64(2), object(10)
memory usage: 

In [30]:
# get the column names

cols = list(alameda_census_rm.columns.values)
cols

['census_tract',
 'rm_no',
 'county_code',
 'county',
 'state_code',
 'census_tract_np',
 'census_tract_period',
 'total_population',
 '62yr_pp_pct',
 'median_age',
 'hispanic_pp_pct',
 'non_hispanic_black_pp_pct',
 '_rm_per_1000']

In [31]:
# reorder the columns

alameda_census_rm = alameda_census_rm[[
    'census_tract', 
    '_rm_per_1000',
    'rm_no',
    'county_code',
    'county',
    'state_code',
    'census_tract_np',
    'census_tract_period',
    'total_population',
    '62yr_pp_pct',
    'median_age',
    'hispanic_pp_pct',
    'non_hispanic_black_pp_pct',
 ]]

In [32]:
# sort by reverse mortgages originated per 1000 people

alameda_census_rm.sort_values(by='_rm_per_1000', ascending=False)

,census_tract,_rm_per_1000,rm_no,county_code,county,state_code,census_tract_np,census_tract_period,total_population,62yr_pp_pct,median_age,hispanic_pp_pct,non_hispanic_black_pp_pct
68,06001421400,6.119951,10,06001,Alameda County,CA,421400,4214,1634,37.5%,54.1,6.4%,1.8%
5,06001409900,5.491488,20,06001,Alameda County,CA,409900,4099,3642,25.5%,48.9,8.1%,47.1%
76,06001425101,5.424955,9,06001,Alameda County,CA,425101,4251.01,1659,33.8%,50.5,5.8%,3.5%
2,06001407700,5.094614,21,06001,Alameda County,CA,407700,4077,4122,19.5%,43.9,16.2%,42.9%
13,06001404300,4.647110,16,06001,Alameda County,CA,404300,4043,3443,22.7%,44.9,12.0%,1.7%
...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,06001435104,0.163479,1,06001,Alameda County,CA,435104,4351.04,6117,11.8%,31.6,38.6%,11.0%
305,06001423602,0.153186,1,06001,Alameda County,CA,423602,4236.02,6528,6.2%,22.2,8.7%,1.3%
286,06001450743,0.149611,1,06001,Alameda County,CA,450743,4507.43,6684,5.4%,32.6,5.8%,5.3%
303,06001441925,0.137155,1,06001,Alameda County,CA,441925,4419.25,7291,5.4%,31,7.5%,6.0%


## export to csv

In [33]:
alameda_rm_name.to_csv('./reverse_mortgage_data/alameda_rm_with_name.csv')
alameda_census_rm.to_csv('./reverse_mortgage_data/alameda_census_rm.csv', index=False)